In [2]:
from pathlib import Path
from ontology_learner.lit_mining.litmining_utils import get_fulltext_from_pmcid_json
import re
import string
from nltk.corpus import stopwords

# %%


In [3]:
datadir = Path('/Users/poldrack/Dropbox/data/ontology-learner/data/json')
datafiles = list(datadir.glob('*.json'))
print(f'found {len(datafiles)} data files')


found 132257 data files


we need to generate a single 

In [4]:

fulltext = {}

def process_fulltext(fulltext):
    # remove all newlines
    fulltext = fulltext.replace('\n', ' ')
    # remove all punctuation
    fulltext = fulltext.translate(str.maketrans('', '', string.punctuation))
    # remove all numbers
    fulltext = re.sub(r'\d+', '', fulltext)
    # remove stopwords
    fulltext = ' '.join([word for word in fulltext.split() if word not in stopwords.words('english')])
    return fulltext

outfile = Path('/Users/poldrack/Dropbox/data/ontology-learner/data/fulltext.txt')
with open(outfile, 'w') as f:
    for datafile in datafiles:
        pmcid = datafile.stem
        fulltext[pmcid] = get_fulltext_from_pmcid_json(pmcid, datadir)
        # processed_text = process_fulltext(fulltext[pmcid])
        f.write(f'{fulltext[pmcid] }\n')
        

# %%


In [5]:
print(f'found {len(fulltext)} fulltext')

found 132257 fulltext


In [6]:
import pickle
with open('/Users/poldrack/Dropbox/data/ontology-learner/data/fulltext.pkl', 'wb') as f:
    pickle.dump(fulltext, f)



llm embeddings

In [7]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

config = PeftConfig.from_pretrained("BrainGPT/BrainGPT-7B-v0.1")
base_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = PeftModel.from_pretrained(base_model, "BrainGPT/BrainGPT-7B-v0.1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/poldrack/micromamba/envs/llama/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [ ]:
import torch
from llm2vec import LLM2Vec

l2v = LLM2Vec.from_pretrained(
    "BrainGPT/BrainGPT-7B-v0.1",
    peft_model_name_or_path="BrainGPT/BrainGPT-7B-v0.1",
    device_map="cuda" if torch.cuda.is_available() else "cpu",
    torch_dtype=torch.bfloat16,
)